# Ähnlichkeitsmaße


In den letzten Teilen hast du die Vektorisierung von Dokumenten mit Bag-of-Words und mit TF/IDF kennengelernt. Damit Machine Learning Methoden richtig funktionieren können, benötigen sie einen *Ähnlichkeitsbegriff*. 

Du könntest natürlich den Abstand der Vektoren (den *Differenzvektor*) als Ähnlichkeitsmaß verwenden. In hochdimensionalen Vektorräumen funktioniert das allerdings nicht besonders gut - stattdessen verwendet man den *Winkel* zwischen den Vektoren. Da die TF/IDF-Vektoren normiert sind, entspricht das auch dem Skalarprodukt.

Die Machine Learning Algorithmen nutzen das Ähnlichkeitsmaß sehr intensiv. Allerdings kannst du auch ohne Machine Learning bereits die Ähnlichkeiten für dich nutzen.

## Daten einladen

Wie gewohnt lädst du die linguistisch analysierten Daten ein:

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT * FROM nlp_articles WHERE datePublished<'2021-01-01' ORDER BY datePublished", 
                 sql, index_col="id", parse_dates=["datePublished"])

## Vektorisierung

Um Ähnlichkeiten ausrechnen zu können, musst du die Daten vektorisieren.

In [ ]:
from spacy.lang.de.stop_words import STOP_WORDS as stop_words
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words, min_df=5)
tfidf_vectors = tfidf_vectorizer.fit_transform(df["nav"])
tfidf_vectors

Durch die Nutzung von `min_df=5` ist die Matrix einigermaßen übersichtlich geblieben!

## Datensatz betrachten

Du hast den `DataFrame` nach dem Datum sortiert, also sollte am Ende auch die Neujahrs-Meldung stehen:

In [ ]:
df.tail()

Mithilfe von `iloc` kannst du auf eine bestimmte *Position* im `DataFrame` zugreifen. Nachdem der `DataFrame` eine Länge von 10.403 hat, hat das letzte Element die Position 10402:

In [ ]:
pd.set_option('display.max_colwidth', None)
df.iloc[10402][["url", "text"]]

## TF/IDF-Vektoren bestimmen

Mit demselben Index (deswegen ist die Nutzung des Index in diesem Fall besser) kannst du nun den TF/IDF-Vektor auslesen:

In [ ]:
tfidf_vectors[10402]

Der Inhalt ist wenig spektakulär und besteht (natürlich!) aus vielen Nullen:

In [ ]:
tfidf_vectors[10402].todense()

## Ähnlichkeit von zwei Dokumenten berechnen

Betrachte nun zusätzlich noch das vorletzte Dokument:

In [ ]:
df.iloc[10401][["url", "text"]]

Mithilfe der Funktion `cosine_similarity` aus dem Paket `sklearn.metrics.pairwise` kannst du den Cosinus des Winkels zwischen den Dokumente ausrechnen. Nachdem die Vektoren keine negativen Werte enthalten, muss der Cosinus auch immer positiv sein:

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(tfidf_vectors[10402], tfidf_vectors[10401])

Bei den normierten Vektoren entspricht der Cosinus dem Skalarprodukt, das du mit `np.dot` berechnen kannst. `np.dot` kann auch ein Matrixprodukt berechnen, daher muss der zweite Vektor transponiert werden:

In [ ]:
import numpy as np
np.dot(tfidf_vectors[10402], tfidf_vectors[10401].T).todense()

## Ähnlichste Dokumente zu Neujahrswünschen finden

Als nächstes versuchst du nun, die ähnlichsten Dokumente zu dem Neujahrsdokument zu finden. Dazu rechnest du die Ähnlichkeit des Dokuments zu allen TF/IDF-Vektoren (der Document-Term-Matrix) aus:

In [ ]:
sim = cosine_similarity(tfidf_vectors[10402], tfidf_vectors)

Und bestimmst den Index mit dem größten Wert:

In [ ]:
sim.argsort()

Das letzte Dokument ist das Nejahrsdokumenst selbst, aber die beiden anderen sind interessant:

In [ ]:
df.iloc[10315][["url", "text"]]

In [ ]:
df.iloc[1114][["url", "text"]]

## Ähnlichkeit als extrem wichtiges Konzept

Der Dokumentenähnlichkeit wirst du immer wieder begegnen, später auch der (semantischen) Wortähnlichkeit. Die Ähnlichkeit ist als *Metrik* im Machine Learning absolut entscheidend!

## Bonus: Die zwei ähnlichsten Dokumente finden

Matrix mit allen Ähnlichkeiten berechnen:

In [ ]:
sim_all = cosine_similarity(tfidf_vectors, tfidf_vectors)

Identische Dokumente vermeiden:

In [ ]:
sim_all[sim_all > 0.9999] = 0

In [ ]:
sim_all.max()

`unravel_index` bestimmt die x- und y-Koordinaten

In [ ]:
(max_a, max_b) = np.unravel_index(np.argmax(sim_all), sim_all.shape)

In [ ]:
df.iloc[max_a][["url", "text"]]

In [ ]:
df.iloc[max_b][["url", "text"]]